In [10]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import os
from skimage.measure import label, regionprops
from sklearn.neighbors import KNeighborsClassifier
# from skimage.morphology import label

In [ ]:
def apply_clahe(img, clahe):
    blured = cv.GaussianBlur(img,(3,3),0,borderType=cv.BORDER_REPLICATE)
    cl1 = clahe.apply(blured)
    return cl1

def gen_mask(img):
    blured1 = cv.GaussianBlur(img,(3,3),0,borderType=cv.BORDER_REPLICATE)
    ret,mask1 = cv.threshold(blured1,0,255,cv.THRESH_OTSU)
    kernel1 = cv.getStructuringElement(cv.MORPH_ELLIPSE,(50,50))
    closed = cv.morphologyEx(mask1, cv.MORPH_CLOSE, kernel1)
    combined = cv.bitwise_and(blured1,blured1,mask = closed)
    blured2 = cv.GaussianBlur(combined,(3,3),0,borderType=cv.BORDER_REPLICATE)
    ret,mask2 = cv.threshold(blured2,0,255,cv.THRESH_OTSU)
    kernel2 = cv.getStructuringElement(cv.MORPH_ELLIPSE,(20,5))
    kernel3 = cv.getStructuringElement(cv.MORPH_ELLIPSE,(25,25))
    mask3 = cv.morphologyEx(mask2, cv.MORPH_OPEN, kernel2)
    final_mask = cv.morphologyEx(mask3, cv.MORPH_OPEN, kernel3)
    return final_mask

def aumentar_bounding_box(imagem, bbox, fator_aumento):
    altura_imagem, largura_imagem = imagem.shape
    x1, y1, x2, y2 = bbox
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
    largura = x2 - x1
    altura = y2 - y1
    nova_largura = largura * fator_aumento
    nova_altura = altura * fator_aumento
    novo_x1 = int(max(0, cx - nova_largura / 2))
    novo_y1 = int(max(0, cy - nova_altura / 2))
    novo_x2 = int(min(largura_imagem, cx + nova_largura / 2))
    novo_y2 = int(min(altura_imagem, cy + nova_altura / 2))
    return novo_x1, novo_y1, novo_x2, novo_y2

def get_iou(bb1, bb2):
    assert bb1[0] <= bb1[2]
    assert bb1[1] <= bb1[3]
    assert bb2[0] <= bb2[2]
    assert bb2[1] <= bb2[3]
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])
    if x_right < x_left or y_bottom < y_top: return 0.0
    intersection_area = (x_right - x_left + 1) * (y_bottom - y_top + 1)
    bb1_area = (bb1[2] - bb1[0] + 1) * (bb1[3] - bb1[1] + 1)
    bb2_area = (bb2[2] - bb2[0] + 1) * (bb2[3] - bb2[1] + 1)
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

def get_bb_lists(img, mask, path, expand):
    h, w, _ = img.shape
    box_list_right = []
    box_list_found = []
    with open(path, "r") as file1:
        for line in file1.readlines():
            class_id, x_center, y_center, width, height = map(float, line.split())
            x_min = int((x_center - width / 2) * w)
            y_min = int((y_center - height / 2) * h)
            x_max = int((x_center + width / 2) * w)
            y_max = int((y_center + height / 2) * h)
            box = (x_min, y_min, x_max, y_max)
            box_list_right.append(box)
    lbl_0 = label(mask) 
    props = regionprops(lbl_0)
    box_list_found = []
    for prop in props:
        box = (prop.bbox[1], prop.bbox[0], prop.bbox[3], prop.bbox[2])
        box = aumentar_bounding_box(mask, box, expand)
        box_list_found.append(box)
    
    #print_boxes(img, box_list_found, box_list_right)

    return box_list_found, box_list_right

def print_boxes(img, box_list_found, box_list_right):
    right = cv.GaussianBlur(img,(3,3),0,borderType=cv.BORDER_REPLICATE)
    mine = cv.GaussianBlur(img,(3,3),0,borderType=cv.BORDER_REPLICATE)
    for box in box_list_right:
        cv.rectangle(right, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
    for box in box_list_found:
        cv.rectangle(mine, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)
    
    fig = plt.figure(figsize =(12,10)) 

    plt.subplot(1, 2, 1)
    plt.imshow(right, vmin=0, vmax=255)
    plt.title('Bounding boxes corretas')

    plt.subplot(1, 2, 2)
    plt.imshow(mine, vmin=0, vmax=255)
    plt.title('Minhas bouding boxes')

def measure_accuracy(box_list_found, box_list_right):
    box_list_found = np.array(box_list_found)
    box_list_right = np.array(box_list_right)
    lf_size, lr_size = len(box_list_found), len(box_list_right)
    if len(box_list_right) > 0:
        classes = np.arange(len(box_list_right))
        knn = KNeighborsClassifier(n_neighbors=1)
        knn.fit(box_list_right, y=classes)
        matches = knn.predict(box_list_found)
        total = 0
        for i in range(lf_size):
            j = matches[i]
            current = get_iou(box_list_found[i], box_list_right[j])
            total += current/lf_size
    else: 
        total = 1
    mod = max(0,1-abs(lr_size-lf_size)*0.01)
    total *= mod
    #print("Acurácia para essa imagem foi de: {:.2f}%".format(total*100))
    return total

In [12]:
TRAIN_PATH = './aquarium_pretrain/train'
VAL_PATH = './aquarium_pretrain/valid'
TEST_PATH = './aquarium_pretrain/test'
NEW_PATH = './aquarium_pretrain/new'

TRAIN_LABELS = TRAIN_PATH + '/labels'
VAL_LABELS = VAL_PATH + '/labels'
TEST_LABELS = TEST_PATH + '/labels'
NEW_LABELS = NEW_PATH + '/labels'

TRAIN_IMAGES = TRAIN_PATH + '/images'
VAL_IMAGES = VAL_PATH + '/images'
TEST_IMAGES = TEST_PATH + '/images'
NEW_IMAGES = NEW_PATH + '/images'

clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

In [ ]:
def main(folder, expand):
    if folder == 'train': image_dir, label_dir = TRAIN_IMAGES, TRAIN_LABELS
    elif folder == 'valid': image_dir, label_dir = VAL_IMAGES, VAL_LABELS
    elif folder == 'test': image_dir, label_dir = TEST_IMAGES, TEST_LABELS
    elif folder == 'new': image_dir, label_dir = NEW_IMAGES, NEW_LABELS
    image_files = sorted(os.listdir(image_dir))
    number_img = len(image_files)
    total_accuracy = 0
    for image_file in image_files:
        img_path = os.path.join(image_dir, image_file)
        label_path = os.path.join(label_dir, image_file[:-4] + '.txt')
        img = cv.imread(img_path)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
        enhanced = apply_clahe(gray, clahe)
        mask = gen_mask(enhanced)
        box_list_found, box_list_right = get_bb_lists(img, mask, label_path, expand)
        accuracy = measure_accuracy(box_list_found, box_list_right)
        total_accuracy += accuracy/number_img
    print("Acurácia para o conjunto de imagens {} foi de: {:.2f}%".format(folder, total_accuracy*100))

In [ ]:
value = 1.2
main('train', value)
main('valid', value)
main('test', value)
main('new', value)